# Generating Text using Markov Chains

### Aim:
In this notebook, we aim to generate similar storyline like Harry Potter by building a 1st Order Markov Chains around all the books of Harry Potter at word level. To know in detail the working of the model read the related blog here.

### Author:
1. [Prakhar Mishra](https://www.linkedin.com/in/prakhar21/)

__For more such materials follow me on __[Medium](https://medium.com/@prakhar.mishra)

### Resources:
1. [Markov Chain explained Visually](http://setosa.io/ev/markov-chains/)
2. [Markov Chains in Python](https://www.datacamp.com/community/tutorials/markov-chains-python-tutorial)
3. [Markov Chains (YouTube)](https://www.youtube.com/watch?v=uvYTGEZQTEs)

### Improvement Scope
1. Try higher order markov chains (maybe 2 or 3)
2. Try increasing vocabulary of words in the current one.
3. Try tuning the exploration factor i.e. randomness_level

### Importing Libraries

In [132]:
import collections
import itertools
import operator
import codecs
import random
import nltk
import glob
import os
import re

### Class that encapsulates all the functionality

In [133]:
class TextGenerator(object):
    
    def __init__(self, data_list):
        self.text = self._load(data_list)
        self.text_tokens = self._prune(self._tokenize())
        self.states = list(set(self.text_tokens))
        self.possible_transitions = self._get_transitions()
        self.trasnsition_probabilites = self.train()
        self.total_words = 0.0
        
    def _load(self, files):
        text = " "
        for f in files:
            print 'Reading {}'.format(f)
            with codecs.open(f, 'rb', 'utf-8') as infile:
                text += self._clean(infile.read().encode('utf-8').decode('ascii', 'ignore')).strip()
        return text
    
    def _get_possibilities(self, state):
        words = []
        for index, value in enumerate(self.text_tokens):
            if value == state:
                try:
                    words.append(self.text_tokens[index+1])
                except:
                    words.append('EOS')
        return {state: dict(collections.Counter(words))}
    
    def _add_probabilities(self, possibilities):
        temp = {}
        for possibility in possibilities:
            for k, v in possibility.items():
                temp[k] = [{'probab': (count/float(self.total_words)) * (1/float(len(v))), 'word': wrd} for wrd,count in v.items()]
        return temp
    
    def train(self):
        possibilities = []
        for state in self.states:
            possibilities.append(self._get_possibilities(state))
        probabilities = self._add_probabilities(possibilities)
        return probabilities
            
    def _get_transitions(self):
        return [[self.states] for state in self.states]
    
    def _prune(self, tokens):
        if len(tokens) > 100000:
            self.total_words = 100000
            return tokens[:self.total_words]
        self.total_words = len(tokens)
        return tokens
        
    def _clean(self, text):
        text = text.lower()
        text = re.sub(r"(\n|\t|/)", " ", text)
        text = re.sub(r'([.,/#!$%^&*;:{}=_`~()-])[.,/#!$%^&*;:{}=_`~()-]+', r'\1', text)
        text = re.sub('([.,!?()])', r' \1 ', text)
        return re.sub(r"\s{2,}", " ", text)
    
    def get_len(self, d):
        return len(d)
    
    def _tokenize(self):
        tokens = nltk.word_tokenize(self.text)
        return tokens

### Loading and Calculating Transition Probabilities

In [134]:
# preparing the data for training the model

data = os.path.abspath('data')
files = glob.glob(data+'/*')

generator = TextGenerator(files)

# length of the text
print "Length of the training file {}".format(generator.get_len(generator.text))
print "Number of words {}".format(generator.get_len(generator.text_tokens))
print generator.states[:15]

Reading /home/prakhar/blogs/markov_tg/data/Book 1 - The Philosopher's Stone.txt
Reading /home/prakhar/blogs/markov_tg/data/Book 7 - The Deathly Hallows.txt
Reading /home/prakhar/blogs/markov_tg/data/Book 6 - The Half Blood Prince.txt
Reading /home/prakhar/blogs/markov_tg/data/Book 4 - The Goblet of Fire.txt
Reading /home/prakhar/blogs/markov_tg/data/Book 2 - The Chamber of Secrets.txt
Reading /home/prakhar/blogs/markov_tg/data/Book 3 - The Prisoner of Azkaban.txt
Reading /home/prakhar/blogs/markov_tg/data/Book 5 - The Order of the Phoenix.txt
Length of the training file 6592904
Number of words 100000
[u'wrought-iron', u'yellow', u'four', u'woods', u'spiders', u'hanging', u'humming', u'wizardry', u'lord', u'flicking', u'three-thirty', u'sinking', u'figg', u'foul', u'screaming']


### Generating Story using the patterns observed from Corpus

In [135]:
# test the model

def formatter(s):
    s = s.split()
    # greedy sentence finisher (matches to last (.))
    s = ' '.join(s[:[idx for idx, ch in enumerate(s) if ch == '.'][-1]+1])
    s = s.capitalize()  # sentence casing
    s = re.sub(r'\s(\.|,|!|\?|\(|\)|\]|\[)', r'\1', s) # remove padded space before punc.
    return s

seed_word = 'phoenix'
story = [seed_word]
words = 0
max_words = 100
randomness_level = 3

while words < max_words-1:
    words += 1    
    candidates = generator.trasnsition_probabilites.get(seed_word)
    temp = sorted(candidates, reverse=True)
    candidates = [i.get('probab') for i in temp]
    grouped = sum([i[1] for i in [(k, sum(1 for i in g)) for k,g in itertools.groupby(candidates)][:randomness_level]])
    seed_word = random.choice(temp[:grouped]).get('word')
    story.append(seed_word)
    
print formatter(' '.join(story).encode('utf-8').decode('ascii', 'ignore'))

Phoenix feather duster at all right. he had a bit more to be in his face. rowling he was going to the other, said ron and a few feet, but he said ron and the philosophers stone - j. he said harry potter and the philosophers stone - j. he had a large ice cream on the door and a bit, but i know, and hermione. he had been to the door, and hermione.
